In [22]:
import pandas as pd
import calendar
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import figure
plt.style.use("ggplot")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from matplotlib import pyplot
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LinearRegression


In [21]:
df = pd.read_excel('C:\\Users\\paulo.almeida\\Downloads\\archive (1)\\Data_Train_m.xlsx')
df.dtypes

Airline             object
Date_of_Journey     object
Source              object
Destination         object
Route               object
Dep_Time            object
Arrival_Time        object
Duration             int64
Total_Stops        float64
Additional_Info     object
Price                int64
dtype: object

In [3]:
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR DEL,22:20,01:10 22 Mar,250,0.0,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU IXR BBI BLR,05:50,13:15,725,2.0,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL LKO BOM COK,09:25,04:25 10 Jun,19,2.0,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU NAG BLR,18:05,23:30,525,1.0,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR NAG DEL,16:50,21:35,445,1.0,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU BLR,19:55,22:25,230,0.0,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU BLR,20:45,23:20,235,0.0,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR DEL,08:20,11:20,3,0.0,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR DEL,11:30,14:10,240,0.0,No info,12648


In [4]:
df['Dep_time_slice'] = df['Dep_Time'].str.slice(stop=2)
df['Dep_time_slice'] = pd.to_numeric(df['Dep_time_slice'], downcast = 'float')

df['Arrival_Time_slice'] = df['Arrival_Time'].str.slice(stop=2)
df['Arrival_Time_slice'] = pd.to_numeric(df['Arrival_Time_slice'], downcast = 'float')

#df['Date_of_Journey'] = df['Dep_Time'].str.slice(stop=2)
#df['Dep_time_slice'] = pd.to_numeric(df['Dep_time_slice'], downcast = 'float')


df['Route'] = df['Route'].str.replace(' ', '')

In [5]:
df['Day_journey'] = df['Date_of_Journey'].str.slice(stop=2)
df['Day_journey'] = df['Day_journey'].str.replace('/', '')
df['Day_journey'] = pd.to_numeric(df['Day_journey'], downcast = 'float')
df['Month_journey'] = df['Date_of_Journey'].str.slice(start=3, stop=-5)
df['Month_journey'] =pd.to_numeric(df['Month_journey'], downcast='unsigned')
df['Month_journey'] =df['Month_journey'].astype('Int64')
df['Month_journey'] = df['Month_journey'].apply(lambda x: calendar.month_name[x])
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Dep_time_slice,Arrival_Time_slice,Day_journey,Month_journey
0,IndiGo,24/03/2019,Banglore,New Delhi,BLRDEL,22:20,01:10 22 Mar,250,0.0,No info,3897,22.0,1.0,24.0,March
1,Air India,1/05/2019,Kolkata,Banglore,CCUIXRBBIBLR,05:50,13:15,725,2.0,No info,7662,5.0,13.0,1.0,May
2,Jet Airways,9/06/2019,Delhi,Cochin,DELLKOBOMCOK,09:25,04:25 10 Jun,19,2.0,No info,13882,9.0,4.0,9.0,June
3,IndiGo,12/05/2019,Kolkata,Banglore,CCUNAGBLR,18:05,23:30,525,1.0,No info,6218,18.0,23.0,12.0,May
4,IndiGo,01/03/2019,Banglore,New Delhi,BLRNAGDEL,16:50,21:35,445,1.0,No info,13302,16.0,21.0,1.0,March


In [6]:
def time_day(x):
    if 0<x<=6:
        return 0
    elif 6<x<=12:
        return 1
    elif 12<x<=18:
        return 2
    elif 18<x<=24:
        return 3
    elif 24<x<=31:
        return 4
   
      

def time_bin(x):
   
    if 0<x<=6:
        return 0
    elif 6<x<=12:
        return 1
    elif 12<x<=18:
        return 2
    elif 18<x<=24:
        return 3
        

In [7]:
#df['Dep_Time'] =pd.to_datetime(df['Dep_Time'])


In [8]:
df['Dep_time_bin']=0
df['Arrival_Time_bin'] = 0
df['Day_journey_bin'] = 0


for i in range (len(df)):
    a = df['Dep_time_slice'][i] 
    b = time_bin(a)
    df['Dep_time_bin'][i] = b
    
    c = df['Arrival_Time_slice'][i]
    d = time_bin(c)
    df['Arrival_Time_bin'] = d
    
    e = df['Day_journey'][i] 
    f = time_day(e)
    df['Day_journey_bin'][i] = f
    

<ipython-input-8-c508d8e99f14>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dep_time_bin'][i] = b
<ipython-input-8-c508d8e99f14>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Day_journey_bin'][i] = f


In [9]:
df1 = df.copy()

In [10]:
df1 = df1.drop(['Date_of_Journey', 'Dep_Time' , 'Arrival_Time', 'Arrival_Time_bin'], axis =1)

df1 = df1.drop(['Dep_time_slice','Arrival_Time_slice','Day_journey'], axis =1)

In [11]:
numeric_features = ['Duration', 'Total_Stops','Dep_time_bin', 'Day_journey_bin']

categorical_features = ['Airline', 'Source' , 'Destination', 'Route', 'Additional_Info', 'Month_journey']




In [12]:
corr=df1[numeric_features].corr()


In [13]:
df_target1 = df1['Price']
df_target = pd.DataFrame(df_target)
df_source= df1.drop(['Price'], axis = 1)

df_dummy = pd.get_dummies(df_source[categorical_features])
df_num = df_source[numeric_features]

df_source = df_dummy.merge(df_num , left_index=True, right_index=True )

source_train, source_test, target_train, target_test = train_test_split(
    df_source, df_target, train_size = 0.7, random_state = 10)
source_train

NameError: name 'df_target' is not defined

In [14]:
df_source_train = source_train.drop(['Duration', 'Total_Stops','Dep_time_bin', 'Day_journey_bin'], axis=1).reset_index().drop(['index'],axis=1)
df_source_test = source_test.drop(['Duration', 'Total_Stops','Dep_time_bin', 'Day_journey_bin'], axis=1).reset_index().drop(['index'],axis=1)
df_source_test

NameError: name 'source_train' is not defined

In [745]:
scaler_minmax = MinMaxScaler()
df_source_sc = scaler_minmax.fit_transform(source_train[numeric_features])
df_source_f = pd.DataFrame(df_source_sc, columns = numeric_features)

df_target_sc = scaler_minmax.fit_transform(target_train) 
df_target_f = pd.DataFrame(df_target_sc, columns = ['Price'])

df_source_f
#

,Duration,Total_Stops,Dep_time_bin,Day_journey_bin
0,0.28,0.25,1.00,0.00
1,0.18,0.25,1.00,0.75
2,0.45,0.50,0.00,0.00
3,0.53,0.25,1.00,0.75
4,0.18,0.25,0.33,0.00
...,...,...,...,...
7473,0.36,0.25,0.33,0.00
7474,0.25,0.25,0.33,0.00
7475,0.05,0.00,1.00,0.75
7476,0.22,0.25,0.33,0.50


In [747]:

df_source_test_sc = scaler_minmax.fit_transform(source_test[numeric_features])
df_source_test_f = pd.DataFrame(df_source_test_sc, columns = numeric_features)

df_target_test_sc = scaler_minmax.fit_transform(target_test) 
df_target_test_f = pd.DataFrame(df_target_test_sc, columns = ['Price'])
df_source_test_f

,Duration,Total_Stops,Dep_time_bin,Day_journey_bin
0,0.30,0.33,0.67,0.25
1,0.56,0.33,1.00,0.50
2,0.00,0.33,1.00,0.00
3,0.23,0.33,0.33,0.75
4,0.28,0.33,0.33,0.25
...,...,...,...,...
3200,0.49,0.67,0.33,0.75
3201,0.61,0.67,0.33,0.00
3202,0.11,0.33,0.67,0.25
3203,0.41,0.33,1.00,1.00


In [748]:
df_source_train = df_source_train.merge(df_source_f, left_index=True, right_index=True)
df_source_train

,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy,Source_Banglore,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi,Route_BLRAMDDEL,Route_BLRBBIDEL,Route_BLRBDQDEL,Route_BLRBOMAMDDEL,Route_BLRBOMBHODEL,Route_BLRBOMDEL,Route_BLRBOMIDRDEL,Route_BLRBOMIDRGWLDEL,Route_BLRBOMIXCDEL,Route_BLRBOMJDHDEL,Route_BLRBOMNAGDEL,Route_BLRBOMUDRDEL,Route_BLRCCUBBIDEL,Route_BLRCCUBBIHYDDEL,Route_BLRCCUBBIHYDVGADEL,Route_BLRCCUDEL,Route_BLRCCUGAUDEL,Route_BLRCOKDEL,Route_BLRDEL,Route_BLRGAUDEL,Route_BLRGOIDEL,Route_BLRHBXBOMAMDDEL,Route_BLRHBXBOMBHODEL,Route_BLRHBXBOMNAGDEL,Route_BLRHYDDEL,Route_BLRHYDVGADEL,Route_BLRIDRDEL,Route_BLRLKODEL,Route_BLRMAADEL,Route_BLRNAGDEL,Route_BLRPNQDEL,Route_BLRSTVDEL,Route_BLRTRVCOKDEL,Route_BLRVGADEL,Route_BLRVGAHYDDEL,Route_BLRVGAVTZDEL,Route_BOMAMDISKHYD,Route_BOMBBIHYD,Route_BOMBDQDELHYD,Route_BOMBHODELHYD,Route_BOMBLRCCUBBIHYD,Route_BOMBLRHYD,Route_BOMCCUHYD,Route_BOMCOKMAAHYD,Route_BOMDEDDELHYD,Route_BOMDELHYD,Route_BOMGOIHYD,Route_BOMGOIPNQHYD,Route_BOMHYD,Route_BOMIDRDELHYD,Route_BOMJAIDELHYD,Route_BOMJDHDELHYD,Route_BOMJDHJAIDELHYD,Route_BOMJLRHYD,Route_BOMMAAHYD,Route_BOMNDCHYD,Route_BOMRPRVTZHYD,Route_BOMUDRDELHYD,Route_BOMVNSDELHYD,Route_CCUAMDBLR,Route_CCUBBIBLR,Route_CCUBBIBOMBLR,Route_CCUBBIHYDBLR,Route_CCUBBIIXRDELBLR,Route_CCUBLR,Route_CCUBOMAMDBLR,Route_CCUBOMBLR,Route_CCUBOMCOKBLR,Route_CCUBOMGOIBLR,Route_CCUBOMHBXBLR,Route_CCUBOMPNQBLR,Route_CCUBOMTRVBLR,Route_CCUDELAMDBLR,Route_CCUDELBLR,Route_CCUDELCOKBLR,Route_CCUDELCOKTRVBLR,Route_CCUDELVGABLR,Route_CCUGAUBLR,Route_CCUGAUDELBLR,Route_CCUGAUIMFDELBLR,Route_CCUHYDBLR,Route_CCUIXABLR,Route_CCUIXBBLR,Route_CCUIXBDELBLR,Route_CCUIXRBBIBLR,Route_CCUIXRDELBLR,Route_CCUIXZMAABLR,Route_CCUJAIBOMBLR,Route_CCUJAIDELBLR,Route_CCUKNUBLR,Route_CCUMAABLR,Route_CCUNAGBLR,Route_CCUPATBLR,Route_CCUPNQBLR,Route_CCURPRHYDBLR,Route_CCUVNSDELBLR,Route_CCUVTZBLR,Route_DELAMDBOMCOK,Route_DELAMDCOK,Route_DELATQBOMCOK,Route_DELBBICOK,Route_DELBDQBOMCOK,Route_DELBHOBOMCOK,Route_DELBLRCOK,Route_DELBOMCOK,Route_DELCCUBOMCOK,Route_DELCOK,Route_DELDEDBOMCOK,Route_DELGOIBOMCOK,Route_DELGWLIDRBOMCOK,Route_DELHYDBOMCOK,Route_DELHYDCOK,Route_DELHYDMAACOK,Route_DELIDRBOMCOK,Route_DELIXCBOMCOK,Route_DELIXUBOMCOK,Route_DELJAIBOMCOK,Route_DELJDHBOMCOK,Route_DELLKOBOMCOK,Route_DELLKOCOK,Route_DELMAABOMCOK,Route_DELMAACOK,Route_DELNAGBOMCOK,Route_DELPNQCOK,Route_DELRPRNAGBOMCOK,Route_DELTRVCOK,Route_DELUDRBOMCOK,Route_MAACCU,Additional_Info_1 Long layover,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight,Month_journey_April,Month_journey_June,Month_journey_March,Month_journey_May,Duration,Total_Stops,Dep_time_bin,Day_journey_bin
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0.28,0.25,1.00,0.00
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.18,0.25,1.00,0.75
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [749]:
df_source_test = df_source_test.merge(df_source_test_f, left_index=True, right_index=True)
df_source_test

,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy,Source_Banglore,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi,Route_BLRAMDDEL,Route_BLRBBIDEL,Route_BLRBDQDEL,Route_BLRBOMAMDDEL,Route_BLRBOMBHODEL,Route_BLRBOMDEL,Route_BLRBOMIDRDEL,Route_BLRBOMIDRGWLDEL,Route_BLRBOMIXCDEL,Route_BLRBOMJDHDEL,Route_BLRBOMNAGDEL,Route_BLRBOMUDRDEL,Route_BLRCCUBBIDEL,Route_BLRCCUBBIHYDDEL,Route_BLRCCUBBIHYDVGADEL,Route_BLRCCUDEL,Route_BLRCCUGAUDEL,Route_BLRCOKDEL,Route_BLRDEL,Route_BLRGAUDEL,Route_BLRGOIDEL,Route_BLRHBXBOMAMDDEL,Route_BLRHBXBOMBHODEL,Route_BLRHBXBOMNAGDEL,Route_BLRHYDDEL,Route_BLRHYDVGADEL,Route_BLRIDRDEL,Route_BLRLKODEL,Route_BLRMAADEL,Route_BLRNAGDEL,Route_BLRPNQDEL,Route_BLRSTVDEL,Route_BLRTRVCOKDEL,Route_BLRVGADEL,Route_BLRVGAHYDDEL,Route_BLRVGAVTZDEL,Route_BOMAMDISKHYD,Route_BOMBBIHYD,Route_BOMBDQDELHYD,Route_BOMBHODELHYD,Route_BOMBLRCCUBBIHYD,Route_BOMBLRHYD,Route_BOMCCUHYD,Route_BOMCOKMAAHYD,Route_BOMDEDDELHYD,Route_BOMDELHYD,Route_BOMGOIHYD,Route_BOMGOIPNQHYD,Route_BOMHYD,Route_BOMIDRDELHYD,Route_BOMJAIDELHYD,Route_BOMJDHDELHYD,Route_BOMJDHJAIDELHYD,Route_BOMJLRHYD,Route_BOMMAAHYD,Route_BOMNDCHYD,Route_BOMRPRVTZHYD,Route_BOMUDRDELHYD,Route_BOMVNSDELHYD,Route_CCUAMDBLR,Route_CCUBBIBLR,Route_CCUBBIBOMBLR,Route_CCUBBIHYDBLR,Route_CCUBBIIXRDELBLR,Route_CCUBLR,Route_CCUBOMAMDBLR,Route_CCUBOMBLR,Route_CCUBOMCOKBLR,Route_CCUBOMGOIBLR,Route_CCUBOMHBXBLR,Route_CCUBOMPNQBLR,Route_CCUBOMTRVBLR,Route_CCUDELAMDBLR,Route_CCUDELBLR,Route_CCUDELCOKBLR,Route_CCUDELCOKTRVBLR,Route_CCUDELVGABLR,Route_CCUGAUBLR,Route_CCUGAUDELBLR,Route_CCUGAUIMFDELBLR,Route_CCUHYDBLR,Route_CCUIXABLR,Route_CCUIXBBLR,Route_CCUIXBDELBLR,Route_CCUIXRBBIBLR,Route_CCUIXRDELBLR,Route_CCUIXZMAABLR,Route_CCUJAIBOMBLR,Route_CCUJAIDELBLR,Route_CCUKNUBLR,Route_CCUMAABLR,Route_CCUNAGBLR,Route_CCUPATBLR,Route_CCUPNQBLR,Route_CCURPRHYDBLR,Route_CCUVNSDELBLR,Route_CCUVTZBLR,Route_DELAMDBOMCOK,Route_DELAMDCOK,Route_DELATQBOMCOK,Route_DELBBICOK,Route_DELBDQBOMCOK,Route_DELBHOBOMCOK,Route_DELBLRCOK,Route_DELBOMCOK,Route_DELCCUBOMCOK,Route_DELCOK,Route_DELDEDBOMCOK,Route_DELGOIBOMCOK,Route_DELGWLIDRBOMCOK,Route_DELHYDBOMCOK,Route_DELHYDCOK,Route_DELHYDMAACOK,Route_DELIDRBOMCOK,Route_DELIXCBOMCOK,Route_DELIXUBOMCOK,Route_DELJAIBOMCOK,Route_DELJDHBOMCOK,Route_DELLKOBOMCOK,Route_DELLKOCOK,Route_DELMAABOMCOK,Route_DELMAACOK,Route_DELNAGBOMCOK,Route_DELPNQCOK,Route_DELRPRNAGBOMCOK,Route_DELTRVCOK,Route_DELUDRBOMCOK,Route_MAACCU,Additional_Info_1 Long layover,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight,Month_journey_April,Month_journey_June,Month_journey_March,Month_journey_May,Duration,Total_Stops,Dep_time_bin,Day_journey_bin
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0.30,0.33,0.67,0.25
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.56,0.33,1.00,0.50
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [750]:
df_source_train = df_source_train.replace([np.inf, -np.inf], np.nan)
df_target_f = df_target_f.replace([np.inf, -np.inf], np.nan)

df_source_test = df_source_test.replace([np.inf, -np.inf], np.nan)
df_target_test_f = df_target_test_f.replace([np.inf, -np.inf], np.nan)

df_source_train = df_source_train.fillna(0)
df_source_test = df_source_test.fillna(0)
df_target_test_f = df_target_test_f.fillna(0)
#df_source_f_train.isnull().sum()

In [751]:
len(df_target_f)

7478

In [784]:
reg = LinearRegression().fit(df_source_train, df_target_f)

reg.score(df_source_test, df_target_f)
    


ValueError: Found input variables with inconsistent numbers of samples: [7478, 3205]

In [778]:



model_rf = RandomForestRegressor(n_estimators=2150, 
                                  random_state=10, 
                                  
                                  #class_weight= 'unbalanced', 
                                  max_features=0.5, 
                                  min_samples_leaf=2, 
                                  min_samples_split=2)

model_rf.fit(df_source_train, df_target_f)

<ipython-input-778-e916cbcac01d>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf.fit(df_source_train, df_target_f)


RandomForestRegressor(max_features=0.5, min_samples_leaf=2, n_estimators=250,
                      random_state=10)

In [779]:
len(df_target_test_f)

3205

In [780]:
#print("Accuracy: ", model_rf.score(df_source_f_train, df_target_f))


print("Accuracy: ", model_rf.score(df_source_test, df_target_test_f))

Accuracy:  0.4686126273650365


In [761]:

feature_importances = pd.DataFrame(model_rf.feature_importances_,
                                   index = df_source_test.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)


feature_importances

,importance
Total_Stops,0.29
Duration,0.15
Airline_Jet Airways,0.08
Day_journey_bin,0.07
Airline_Jet Airways Business,0.06
Additional_Info_In-flight meal not included,0.05
Month_journey_March,0.03
Airline_IndiGo,0.03
Additional_Info_No info,0.03
Additional_Info_Business class,0.02


In [786]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [789]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(df_source_train, df_target_f)


preds = xg_reg.predict(df_source_test)

[20:59:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[20:59:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


In [796]:
rmse = np.sqrt(mean_squared_error(df_target_test_f, preds))
print("RMSE: %f" % (rmse))
xg_reg.score(df_target_test_f, preds)

RMSE: 0.127840


ValueError: feature_names mismatch: ['Airline_Air Asia', 'Airline_Air India', 'Airline_GoAir', 'Airline_IndiGo', 'Airline_Jet Airways', 'Airline_Jet Airways Business', 'Airline_Multiple carriers', 'Airline_Multiple carriers Premium economy', 'Airline_SpiceJet', 'Airline_Trujet', 'Airline_Vistara', 'Airline_Vistara Premium economy', 'Source_Banglore', 'Source_Chennai', 'Source_Delhi', 'Source_Kolkata', 'Source_Mumbai', 'Destination_Banglore', 'Destination_Cochin', 'Destination_Delhi', 'Destination_Hyderabad', 'Destination_Kolkata', 'Destination_New Delhi', 'Route_BLRAMDDEL', 'Route_BLRBBIDEL', 'Route_BLRBDQDEL', 'Route_BLRBOMAMDDEL', 'Route_BLRBOMBHODEL', 'Route_BLRBOMDEL', 'Route_BLRBOMIDRDEL', 'Route_BLRBOMIDRGWLDEL', 'Route_BLRBOMIXCDEL', 'Route_BLRBOMJDHDEL', 'Route_BLRBOMNAGDEL', 'Route_BLRBOMUDRDEL', 'Route_BLRCCUBBIDEL', 'Route_BLRCCUBBIHYDDEL', 'Route_BLRCCUBBIHYDVGADEL', 'Route_BLRCCUDEL', 'Route_BLRCCUGAUDEL', 'Route_BLRCOKDEL', 'Route_BLRDEL', 'Route_BLRGAUDEL', 'Route_BLRGOI

In [795]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, random_state=7)

results = cross_val_score(xg_reg, df_source_train, df_target_f, cv=kfold)

y_test_pred = xg_reg.predict(df_source_test)

mse = mean_squared_error(y_test_pred, df_target_test_f)

y_pred = xg_reg.predict(df_source_test)
a = pd.DataFrame(y_pred)
mse

C:\Users\paulo.almeida\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


[21:10:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[21:10:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[21:10:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[21:10:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[21:10:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[21:10:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:17

0.016343190202291514